# Curso: Bioestatística — Experimento de Regressão Linear Múltipla
## Autores: Sandro da Silva Camargo e Fernando Cardoso

**Problema**:Um experimento foi realizado para testar o efeito
das Variáveis independentes, temperatura média do perı́odo (ºC),
precipitação média (mm), pragas (insetos/dm2 ) e densidade
(plantas/m2 ), na Variável resposta: produção estacional de matéria seca (MS)
(ton/ha).

A base de dados está disponível [aqui](https://github.com/Sandrocamargo/biostatistics/blob/master/datasets/rl-rlm.txt).

Abra este código no seu google colab [clicando aqui](https://colab.research.google.com/github/Sandrocamargo/biostatistics/blob/master/python/bioe_06_Regressao_Linear_Multipla.ipynb).

# Importação de bibliotecas

In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression

# Carga e inspeção dos dados

In [ ]:
dados = pd.read_csv("https://raw.githubusercontent.com/Sandrocamargo/biostatistics/refs/heads/master/datasets/rl-rlm.txt", sep="\t", header=0)
dados.info()
dados.head()

# Ajuste do modelo de regressão linear múltipla

In [ ]:
# === 2. Modelo completo ===
modelo = smf.ols("ms ~ temp + prec + prag + dens", data=dados).fit()
print("=== Modelo completo ===")
print(modelo.summary())

**📊 Resumo geral do modelo**

|Métrica	|Valor	|Interpretação|
|:--|:--|:--|
|R² |= 0.870	|O modelo explica 87% da variação total da matéria seca. Excelente poder explicativo.	|
|R² ajustado| = 0.835	|Considerando o número de variáveis, o modelo ainda mantém bom ajuste.	|
|F(4,15)| = 25.00 p = 1.74e−06	|O conjunto das variáveis explica significativamente a variação em ms. Há relação global significativa.	|


**✅ Conclusão geral**: o modelo como um todo é altamente significativo.

🔹 Coeficientes individuais
|Variável	|Coef.	|p-valor	|Interpretação|
|:--|:--|:--|:--|
|Intercepto |= 0.7196	|—	|—	|Valor médio esperado de ms quando todas as variáveis preditoras são zero (sem significado prático direto).|
|temp |= -0.0129	|0.517	|Não significativo. Temperatura não tem efeito linear detectável sobre a matéria seca.	|
|prec |= 0.0043	|0.001|	✅ Significativo. Aumento da precipitação está associado a aumento da matéria seca; cada unidade de chuva aumenta 0.0043 ton/ha, em média.	|
|prag |= 0.0083	|0.636	|Não significativo. O efeito de pragas não é relevante estatisticamente.	|
|dens |= 0.0486	|0.018	|✅ Significativo. Densidade (provavelmente de plantas) tem efeito positivo sobre a matéria seca; cada unidade adicional aumenta 0.0486 ton/ha, em média.	|

➡️ Somente “precipitação” e “densidade” são significativas. As demais (temperatura e pragas) não contribuem de forma relevante para explicar ms.

🔹 Diagnóstico do modelo
|Teste / Estatística	|Valor	|Interpretação|
|:--|:--|:--|
|Durbin–Watson |= 1.347	|Indica leve autocorrelação positiva dos resíduos, mas não crítica em amostra pequena.	|
|Omnibus / Jarque–Bera |(p > 0.2)	|Sem evidências de violação da normalidade dos resíduos.	|
|Cond. No. |= 3.07e+03	|🚨 Alerta de multicolinearidade. Algumas variáveis (ex.: temperatura, precipitação e densidade) podem estar correlacionadas entre si, o que reduz a precisão dos coeficientes individuais.

**✅ Conclusão interpretativa**
* O modelo geral é estatisticamente significativo (p < 0.001) e explica 87% da variação da matéria seca.

Entre as quatro variáveis preditoras:
* Precipitação e densidade têm efeitos positivos e significativos sobre ms.
* Temperatura e pragas não apresentam efeito significativo.
* A multicolinearidade pode estar inflando erros-padrão, recomenda-se verificar o VIF (Variance Inflation Factor).
* Assim, um modelo reduzido com apenas prec e dens provavelmente manterá alto R² com maior parcimônia e estabilidade.

# Análise de variância

In [ ]:
anova_modelo = sm.stats.anova_lm(modelo, typ=2)
print("\n=== ANOVA do modelo completo ===")
print(anova_modelo)

🔹 1. Interpretação geral

* O modelo geral é significativo (ver resumo OLS anterior, F = 25, p < 0.001).
* A decomposição mostra que precipitação (prec) e densidade (dens) são os fatores que realmente contribuem para explicar ms.
* Temperatura (temp) e pragas (prag) não têm efeito significativo no modelo, sugerindo que podem ser excluídas em um modelo reduzido.

🔹 2. Proporção da variação explicada

* A soma de quadrados total (SQTrat + SQResidual) é:
$SQ_{Total} = SQ_{temp} + SQ_{prec} + SQ_{prag} + SQ_{dens} + SQ_{residual}$

$SQ_{Total} = 0.0179 + 0.6899 + 0.0095 + 0.2896 + 0.6101 ≈ 1.617$

* SQResidual = 0.6101 → cerca de 37,7% da variação não é explicada.

* SQTratamentos (prec + dens + temp + prag) = 1.0079 → cerca de 62,3% da variação total é explicada pelos preditores.
* Isso está de acordo com o R² ≈ 0.87 (considerando escala dos quadrados e ajustes).

🔹 3. Conclusão

* Precipitação (prec): efeito altamente significativo — cada aumento de 1 unidade aumenta significativamente a produção de matéria seca.
* Densidade (dens): efeito positivo e significativo.
* Temperatura (temp) e pragas (prag): não significativas — podem ser consideradas para remoção em um modelo reduzido.
* Resíduos: ainda existe variação não explicada, indicando que outros fatores não incluídos no modelo podem influenciar a matéria seca.

**Em termos práticos: para otimizar a matéria seca, é mais importante focar em precipitação (ou irrigação equivalente) e densidade do plantio do que em temperatura ou pragas, dentro das condições estudadas.**

# Avaliação dos coeficientes do modelo de regressão

In [ ]:
# === 3. Gráficos dos coeficientes ===
coefs = modelo.params[1:]  # ignora o intercepto

plt.figure()
plt.bar(coefs.index, coefs.values)
plt.title("Coeficientes do Modelo")
plt.xlabel("Atributos")
plt.ylabel("Coeficientes")
plt.savefig("mlr-coefs.pdf")
plt.show()

plt.figure()
plt.bar(coefs.index, coefs.values * dados.iloc[:, 1:5].mean().values)
plt.title("Coeficientes × Médias das Variáveis")
plt.xlabel("Atributos")
plt.savefig("mlr-coefs-aj.pdf")
plt.show()

# Avaliação do desempenho do modelo

In [ ]:
plt.figure()
plt.scatter(dados["ms"], modelo.fittedvalues)
plt.plot([dados["ms"].min(), dados["ms"].max()],
         [dados["ms"].min(), dados["ms"].max()], color="red")
plt.xlabel("Matéria Seca Real (ton/ha)")
plt.ylabel("Matéria Seca Predita (ton/ha)")
plt.title(f"R² ajustado = {modelo.rsquared_adj:.4f}")
plt.savefig("mlr-results.pdf")
plt.show()

# Avaliação de modelos reduzidos (com menos variáveis)

In [ ]:
modelos = {
    "modelo2": "ms ~ prec + dens",
    "modelo3": "ms ~ temp + prag + dens",
    "modelo4": "ms ~ temp + prag"
}

for nome, formula in modelos.items():
    print(f"\n=== {nome.upper()} ===")
    mod = smf.ols(formula, data=dados).fit()
    print(mod.summary())
    sm.stats.anova_lm(mod, typ=2)

    plt.scatter(dados["ms"], mod.fittedvalues)
    plt.plot([dados["ms"].min(), dados["ms"].max()],
             [dados["ms"].min(), dados["ms"].max()], color="red")
    plt.xlabel("Matéria Seca Real (ton/ha)")
    plt.ylabel("Matéria Seca Predita (ton/ha)")
    plt.title(f"{nome.upper()} - R² ajustado = {mod.rsquared_adj:.4f}")
    plt.show()

# Criação stepwise de modelos

In [ ]:
print("\n=== Seleção stepwise (grau 1) ===")

# Converte dados para formato sklearn
X = dados[["temp", "prec", "prag", "dens"]]
y = dados["ms"]

lr = LinearRegression()

# Stepwise forward e backward
sfs = SFS(lr,
          k_features="best",
          forward=True,
          floating=True,
          scoring="r2",
          cv=0)
sfs = sfs.fit(X, y)
print(f"Melhores variáveis (stepwise grau 1): {sfs.k_feature_names_}")

# Criação de modelos polinomiais

In [ ]:

dados["temp2"] = dados["temp"]**2
dados["prec2"] = dados["prec"]**2
dados["prag2"] = dados["prag"]**2
dados["dens2"] = dados["dens"]**2

dados["temp3"] = dados["temp"]**3
dados["prec3"] = dados["prec"]**3
dados["prag3"] = dados["prag"]**3
dados["dens3"] = dados["dens"]**3

dados["temp4"] = dados["temp"]**4
dados["prec4"] = dados["prec"]**4
dados["prag4"] = dados["prag"]**4
dados["dens4"] = dados["dens"]**4

model_formulas = [
    "ms ~ temp + prec + prag + dens + I(temp**2) + I(prec**2) + I(prag**2) + I(dens**2)",
    "ms ~ temp + prec + prag + dens + I(temp**2) + I(prec**2) + I(prag**2) + I(dens**2) + I(temp**3) + I(prec**3) + I(prag**3) + I(dens**3)",
    "ms ~ temp + prec + prag + dens + I(temp**2) + I(prec**2) + I(prag**2) + I(dens**2) + I(temp**3) + I(prec**3) + I(prag**3) + I(dens**3) + I(temp**4) + I(prec**4) + I(prag**4) + I(dens**4)"
]

for i, formula in enumerate(model_formulas, start=2):
    print(f"\n=== Stepwise grau {i} ===")
    mod = smf.ols(formula, data=dados).fit()
    print(mod.summary())

    # Seleção automática usando stepwise
    vars_ = ["temp", "prec", "prag", "dens",
             "temp2", "prec2", "prag2", "dens2",
             "temp3", "prec3", "prag3", "dens3",
             "temp4", "prec4", "prag4", "dens4"]
    vars_ = [v for v in vars_ if v in dados.columns]
    X_poly = dados[vars_]
    sfs_poly = SFS(lr, k_features="best", forward=True, floating=True, scoring="r2", cv=0)
    sfs_poly.fit(X_poly, y)
    print(f"Variáveis selecionadas (grau {i}): {sfs_poly.k_feature_names_}")
